In [1]:
# Analyse numérique et tabulaire
import numpy as np
import pandas as pd

# Modélisation et évaluation
import sklearn.cluster as cluster
from sklearn import metrics
import time

# Affichage graphique
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Configurations communes
sns.set_context('poster')
sns.set_color_codes()

In [2]:
# Lire le fichier Parquet
pdf_data = pd.read_parquet("data/user_stats_df_output.parquet")

In [17]:
pdf_data=pdf_data.drop('last_purchase', axis='columns')

In [20]:
# Supprimer les 4 dernières colonnes
pdf_data = pdf_data.iloc[:, :-4]


In [21]:
pdf_data.dtypes

user_id                       int32
number_of_views_2m            int64
number_of_carts_2m            int64
number_of_sessions_2m         int64
count_products_2m             int64
avg_price_2m                float64
number_of_views_5m            int64
number_of_carts_5m            int64
number_of_sessions_5m         int64
count_products_5m             int64
avg_price_5m                float64
number_of_views_7m            int64
number_of_carts_7m            int64
number_of_sessions_7m         int64
count_products_7m             int64
avg_price_7m                float64
days_since_last_purchase      int32
total_purchase_value        float64
cart_abandonments             int64
dtype: object

In [3]:
def plot_clusters(pdf, algorithm, args, kwds):
    ''' Fonction utilitaire de clustering des données contenues dans le DF Pandas <pdf> (2 attributs 'x1' et 'x2')
        à l'aide de l'algorithme <algorithm> avec les arguments internes d'appel <args> et <kwds> pour l'affichage des résultats
        <args> réservé à une liste de paramètres, <kwds> à un dictionnaire de paramètres (kwds pour keywords), <args> avant <kwds>
    '''
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(np.array(pdf))
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.figure(figsize=(6,6))
    plt.scatter(pdf.x1, pdf.x2, c=colors, **PLOT_KWDS)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title("Clusters révélés par {}".format(str(algorithm.__name__)), fontsize=12)
    plt.text(-0.5, 0.65, "Temps d'exécution : {:.2f} s".format(end_time - start_time), fontsize=10)
    plt.show()

In [4]:
def plot_silhouette(lst_clusters, lst_inertie, lst_scores):
    ''' Evolution du score de silhouette en fonction des arguments fournis liste des nb de clusters, inertie et scores calculés
    '''
    # Graphes
    plt.figure(figsize=(20,4))
    plt.plot(lst_clusters, lst_sse, "bo-")
    plt.xticks(lst_clusters); plt.xlabel("Nombre de Clusters", fontsize=10); plt.ylabel("Inertie", fontsize=10)
    plt.show()
    
    plt.figure(figsize=(10,4))
    plt.plot([0]+lst_clusters, np.array(lst_scores), 'bx-')
    plt.xlabel('Nombre de clusters $k$', fontsize=10); plt.ylabel('Silhouette Moyenne', fontsize=10)
    plt.title('La méthode du coude révélant la valeur optimale de $k$', fontsize=12)
    plt.show()

In [5]:
def model_clustering(model, pdf, kwargs):
    # Entrainement du modele de clustering
    mdl = model(**kwargs).fit(pdf)
    # Clusters
    db_labels = mdl.labels_
    labels = mdl.fit_predict(pdf)
    n_clusters_ = len(set(db_labels)) - (1 if -1 in db_labels else 0)
    n_noise_ = list(db_labels).count(-1)
    n_data = len(pdf); ratio =  n_noise_/n_data
    # Scores
    score = metrics.silhouette_score(pdf, labels)
    print("Nombre estimé de clusters: %d" % n_clusters_)
    print("Liste des clusters", set(db_labels))
    print("Nombre estimé de points 'bruités': %d" % n_noise_)
    print("Ratio estimé de points 'bruités': %.2f" % ratio)
    print(f"Score de Silhouette: {score}")

In [10]:
# constante d'initialisation d'échantillonnage
SEED = 123
# dictionnaire pour l'habillage de graphes
PLOT_KWDS = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

In [ ]:
plt.scatter(pdf_data["x1"], pdf_data["x2"], c='b', **PLOT_KWDS)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)

In [8]:
cluster.KMeans

sklearn.cluster._kmeans.KMeans

In [22]:
plot_clusters(pdf_data, cluster.KMeans, (), {'n_clusters':6, 'n_init': 10, 'random_state': SEED})

AttributeError: 'DataFrame' object has no attribute 'x1'

<Figure size 600x600 with 0 Axes>

In [12]:
pdf_data.columns

Index(['user_id', 'number_of_views_2m', 'number_of_carts_2m',
       'number_of_sessions_2m', 'count_products_2m', 'avg_price_2m',
       'number_of_views_5m', 'number_of_carts_5m', 'number_of_sessions_5m',
       'count_products_5m', 'avg_price_5m', 'number_of_views_7m',
       'number_of_carts_7m', 'number_of_sessions_7m', 'count_products_7m',
       'avg_price_7m', 'last_purchase', 'days_since_last_purchase',
       'total_purchase_value', 'cart_abandonments', 'preferred_brand',
       'preferred_category', 'most_active_time', 'most_active_day'],
      dtype='object')